In [1]:
import sys
sys.path = ["/media/anonyman637/CoreProgramming/CProjects/Official/include/Console/ExtractCtrlFunc/pythonVenv/Linux/offCtrlFuncExtrcVenv/lib/python3.6",
        "/media/anonyman637/CoreProgramming/CProjects/Official/include/Console/ExtractCtrlFunc/pythonVenv/Linux/offCtrlFuncExtrcVenv/lib/python3.6/distutils",
        "/media/anonyman637/CoreProgramming/CProjects/Official/include/Console/ExtractCtrlFunc/pythonVenv/Linux/offCtrlFuncExtrcVenv/lib/python3.6/site-packages",
        "/media/anonyman637/CoreProgramming/CProjects/Official/include/Console/ExtractCtrlFunc/pythonVenv/Linux/offCtrlFuncExtrcVenv/lib/python3.6/__pycache__"
        ]
import PyPDF2

## Reading Control Functions from ECMA-48 Std

In [2]:
standardFile = open('Ecma-048.pdf', 'rb')

In [3]:
standardPdf = PyPDF2.PdfFileReader(standardFile)

In [4]:
standardPdf.numPages

108

### Regex for formating text

In [5]:
import re

In [6]:
controlRegex = re.compile(r'8.3.\d+')

In [7]:
pageRegex = re.compile(r'- \d+ -')

In [8]:
text = ""

In [9]:
for pg in range(46, 89):
    text += standardPdf.getPage(pg).extractText()

#### Ommiting Page Numbers

In [10]:
text = re.sub(pageRegex, "", text)

#### Creating List of Control Functions

In [11]:
controlFuncRaw = controlRegex.split(text)

#### Eliminating Juncs

In [12]:
controlFuncRaw = controlFuncRaw[14:]

In [13]:
controlFuncRaw[161] = controlFuncRaw[161][:198]

In [14]:
len(controlFuncRaw)

162

#### More Text Formatting

In [15]:
for i in range(len(controlFuncRaw)):
    controlFuncRaw[i] = controlFuncRaw[i].strip()

#### Getting Syntax and Documentation

In [16]:
# https://regex101.com/r/oWa5tj/2
parseRegex = re.compile(r'''
(.*)                    # Matching the Code
\s+\-\s+                
(.*)                    # Full name
\s+Notation:\s+\(       
(.*)                    # Notation
\)\s+Representation:\s+ 
(.*\/\d\d)              # Representation
\s+                     
(.*Parameter\sdefault\svalues?    # Parameter Info
\:\s?
(.*\=\s?\d+))?          # Defualt Parameter Substitution
(.*)                    # Documentation
''', re.VERBOSE)

In [17]:
print(controlFuncRaw[131])

SPI - SPACING INCREMENT Notation: (Pn1;Pn2) Representation: CSI Pn1;Pn2 02/00 04/07 No parameter default values.  SPI is used to establish the line spacing and the character spacing for subsequent text. The established line spacing remains in effect until the next occurrence of SPI or of SET LINE SPACING (SLS) or of SELECT LINE SPACING (SVS) in the data stream. The established character spacing remains in effect until the next occurrence of SET CHARACTER SPACING (SCS) or of SELECT CHARACTER SPACING (SHS) in the data stream, see annex C.  Pn1 specifies the line spacing  Pn2 specifies the character spacing    The unit in which the parameter values are expressed is that established by the parameter value of SELECT SIZE UNIT (SSU).


In [18]:
parseRegex.findall(controlFuncRaw[131])[0]

('SPI',
 'SPACING INCREMENT',
 'Pn1;Pn2',
 'CSI Pn1;Pn2 02/00 04/07',
 '',
 '',
 'No parameter default values.  SPI is used to establish the line spacing and the character spacing for subsequent text. The established line spacing remains in effect until the next occurrence of SPI or of SET LINE SPACING (SLS) or of SELECT LINE SPACING (SVS) in the data stream. The established character spacing remains in effect until the next occurrence of SET CHARACTER SPACING (SCS) or of SELECT CHARACTER SPACING (SHS) in the data stream, see annex C.  Pn1 specifies the line spacing  Pn2 specifies the character spacing    The unit in which the parameter values are expressed is that established by the parameter value of SELECT SIZE UNIT (SSU).')

In [69]:
nl = "n37n"
def parseFunction(stdText):
    tokens = parseRegex.findall(stdText)
    funcTokens = tokens[0]
    
    # Group 1 contains abbreviation
    abbr = funcTokens[0].strip()
    
    # Group 2 contains full name
    name = funcTokens[1].strip()
    
    # Group 3 contains function args
    ## Get Rid of C0 and C1
    args = funcTokens[2]
    if "C0" in args or "C1" in args:
        args = []
    else:
        ### Replacing ... with blank for Ps... args
        ### Spliting Args for use
        args = args.replace("...", "").split(";")
    
    # Group 4 contains representation
    rep = funcTokens[3]
    ## Macro args will convert to strings
    for arg in args:
        rep = rep.replace(arg, "#" + arg)

    ## Parse the Hex Value Strings
    ### Find the Hex Values
    hexRegexStr = r'(\d{0,2})/(\d{0,2})'
    hexRegex = re.compile(hexRegexStr)
    hexStrs = hexRegex.findall(rep)
    ### Convert them to str
    for i in range(len(hexStrs)):
        hexStrs[i] = "\"" + chr(int(hexStrs[i][0]) * 16 + int(hexStrs[i][1])) + "\""
    
    ### Replacing Hex String with true Hex
    rep = re.sub(hexRegexStr,
                 lambda match: str(hexStrs.pop(0)),
                 rep)
    ### Taking the last representation if there are multiples (divided by 'or')
    ### Replacing ; with ";" because they should be pure strings
    ### Replacing ... with blank for Ps... args
    rep = rep.split("or")[-1].replace(";", " \";\" ").replace("...", "").strip()
    # The all args function is the first representation of all representations
    # The all default args function is the second which we will configure later
    rep = [rep]
    args = [args]
    
    
    # XXX Group 5 contains overall param info XXX
    
    # Group 6 contains default values for some parameters
    ## Multiple Definitions for default param value
    if funcTokens[5] != "":
        defParams = funcTokens[5].replace(" ", "").split(";")
        for i in range(len(defParams)):
            defParams[i] = defParams[i].split("=")[0]
            # By Definition, the absent sent will be provided default values if available
            # So we just omitted the default value args one at a time
            ## The one absent defualt representation and formatting spaces too
            rep.append(re.sub(r"\s{2,}", " ", rep[0].replace("#" + defParams[i], "")))
            ### Creating marks for def args
            defArgMarks = []
            for j in range(len(args[0])):
                if args[0][j] == defParams[i]:
                    defArgMarks.append(j)
        
            targs = args[0]
            ### Deleting the marked index
            for defArgMark in defArgMarks:
                targs = targs[:defArgMark] + targs[defArgMark + 1:]
        
            args.append(targs)
    else:
        defParams = []
    
    
    ## The all absent default representation
    if len(defParams) == len(args[0]) and len(args[0]) > 1:
        # The all default args function is the second which we are configuring now
        rep.append(rep[0])
        args.append(args[0])
        
        for defParam in defParams:
            ### Replacing default args and params with blanks and formatting spaces too
            rep[-1] = re.sub(r"\s{2,}", " ", rep[1].replace("#" + defParam, ""))
            ### marking def args
            defArgMarks = []
            for i in range(len(args[-1])):
                if args[-1][i] == defParam:
                    defArgMarks.append(i)
                    
            ### deleting marked index
            for defArgMark in defArgMarks:
                args[-1] = args[-1][:defArgMark] + args[-1][defArgMark + 1:]
    
    
    # Group 7 contains the documentation
    ## Double or more spaces are new lines
    doc = re.sub(r"\s{2,}", "\n", funcTokens[6].strip()).splitlines()
    ### Formatting the doc
    foldCol = 60
    for li in range(len(doc)):
        line = doc[li]
        if len(line) < foldCol:
            continue
        #### marks for folding cols at suitable word length
        foldMarks = [0]
        for i in range(0, len(line), foldCol):
            # Trying to get the index around 70
            lci = foldMarks[-1]
            lastWordOfCurrentLine = line.rfind(" ", lci, lci + foldCol)
            firstWordOfNextLine = line.find(" ", lci + foldCol)
            minIsLastWord = foldCol - lastWordOfCurrentLine < firstWordOfNextLine - foldCol
            foldMarks.append(lastWordOfCurrentLine if minIsLastWord else firstWordOfNextLine) 
            
#       Tolerance level of last 
        if foldMarks[-1] - foldMarks[-2] < foldCol:
            foldMarks[-1] = None
        else:
            foldMarks.append(None)
        
        foldDoc = [line[foldMarks[i]:foldMarks[i + 1]] for i in range(len(foldMarks) - 1)]
        doc[li] = (nl + " *").join(foldDoc)
    
    doc = (nl * 2 + " * ").join(doc)
    
    funcs = []
    # Creating the function
    ## Sorting, because we appended reversed earlier
    ### All Params + Default Params + All Default Params
    if len(defParams) == len(args[0]) and len(args[0]) > 1:
        rep = [rep[0]] + rep[1:-1][::-1] + [rep[-1]]
        args = [args[0]] + args[1:-1][::-1] + [args[-1]]
    
    for i in range(len(rep)):
        func = "#define "
        if len(args[i]) > 0:
            func = func + "_" * i + abbr + "(" + ", ".join(args[i]) + ")"
        elif len(defParams) > 0:
            func = func + "_" + abbr + "_"
        else:
            func = func + abbr
        func = func + " " + rep[i]
        
        ### Unique Doc for each representation of function
        customDoc = []
        #### abbr and name Info
        customDoc.append(abbr + " - " + name + nl)
        #### Args Info
        customDoc.append("@ Parameters Required: ")
        customDoc.append(", ".join(args[i]) + nl if len(args[i]) > 0 else "No Parameter Required" + nl)
        #### Doc Info
        customDoc.append("@ More Info:")
        customDoc.append(doc)
        #### Joining All
        customDoc = "/*" + nl + " * " + (nl + " * ").join(customDoc) + nl + " *" + nl + " */"
        func = customDoc + nl + func + nl*2
        funcs.append(func)
        
    return funcs

In [70]:
parseFunction(controlFuncRaw[0])

['/*n37n * ACK - ACKNOWLEDGEn37nn37n * @ Parameters Required: n37n * No Parameter Requiredn37nn37n * @ More Info:n37n * ACK is transmitted by a receiver as an affirmative responsen37n * to the sender.n37nn37n * The use of ACK is defined in ISO 1745.n37n *n37n */n37n#define ACK "\x06"n37nn37n']

In [71]:
extracted = open("ctrlFuncCodedCharSet.h", "w", )
i = 1
problems = []
for rawFunc in controlFuncRaw:
    try:
        parsedFuncs = parseFunction(rawFunc)
    except Exception:
        problems.append(i)
        continue
    for func in parsedFuncs:
        func = repr(func)[1:-1]
        extracted.write(func)
        print(func)
    i += 1
extracted.close()

/*n37n * ACK - ACKNOWLEDGEn37nn37n * @ Parameters Required: n37n * No Parameter Requiredn37nn37n * @ More Info:n37n * ACK is transmitted by a receiver as an affirmative responsen37n * to the sender.n37nn37n * The use of ACK is defined in ISO 1745.n37n *n37n */n37n#define ACK "\x06"n37nn37n
/*n37n * APC - APPLICATION PROGRAM COMMANDn37nn37n * @ Parameters Required: n37n * No Parameter Requiredn37nn37n * @ More Info:n37n * to 07/14. The control string is closed by the terminatingn37n * delimiter STRING TERMINATOR (ST). The interpretation ofn37n * the command string depends on the relevant application program.n37n *n37n */n37n#define APC application program use. The command string following may consist of bit combinations in the range "\x08" to "\r" and " "n37nn37n
/*n37n * BEL - BELLn37nn37n * @ Parameters Required: n37n * No Parameter Requiredn37nn37n * @ More Info:n37n * BEL is used when there is a need to call for attention; it mayn37n * control alarm or attention devices.n37n *n37n *

In [25]:
print(i)
print(problems)

162
[108]
